# Running Quantum Cicuits

- Lesson overview
- Hardware – IBM quantum processors
- Software: Qiskit and Qiskit Runtime
- Quantum Circuits
- Designing a quantum circuit: Qiskit patterns
    - Map
    - Optimize
    - Execute
    - Post-process
- Conclusion


## Lesson overview
This lesson will be a high-level look at the basics of running a utility-scale quantum computation, from the quantum hardware used to principles to consider when designing a quantum circuit. Ideally, by the end of this lesson, you'll know:

What IBM Quantum computers actually are. You'll need to know the basics of the hardware features to optimally design your quantum circuits to run on it.

What Qiskit is, what primitives are, and how we can use them to create and execute quantum circuits.

The typical workflow we follow to run experiments at scale. This includes selecting the best primitives for your use-case, mapping a problem to a quantum circuit, and applying error mitigation and suppression, which enable us to squeeze as much power out of these machines as possible.
---
## Hardware – IBM quantum processors
To understand how we can make optimal choices in designing large-scale quantum circuits, we need to know a little bit about the actual hardware that will run these circuits. So let's briefly discuss physical qubits and IBM quantum processors.

IBM quantum processors are built using superconducting transmon qubits, which are electrical circuits composed of a Josephson junction and a capacitor connected in parallel. The Josephson junction is a nonlinear inductor created from two overlapping layers of superconducting metal with an insulating barrier between them. At very low temperatures, the electrons in superconductors pair up to form what is known as a cooper pair. Cooper pairs can spontaneously tunnel through the insulating barrier from one side of the junction to the other. This tunneling behavior gives rise to the nonlinear properties which create our qubit.

Microwave transmission lines are fabricated on the chip to deliver microwave signals to the qubits. When we apply highly calibrated microwave pulses — with specific frequency, amplitude, shape, and duration — to the lines, we can make the qubits do specific things. This forms the basis of our quantum gates. We fabricate the chip so that neighboring qubits are connected in a specific lattice structure called a heavy-hex lattice. This connectivity — the so-called topology — of our processors is an important factor to consider when designing a circuit, as we'll discuss later in the lesson.

The instructions for the microwave pulse go from your computer, through the cloud, and to room-temperature control electronics, which interpret those instructions and physically generate the pulses. After the room-temperature control boxes create the pulses, they travel through cables into a dilution refrigerator and eventually to the quantum chip. The signal goes into the resonators, through a wirebond, and then flows down the transmission line into our qubits.

IBM has dozens of quantum computers around the world, and we've recently upgraded our fleet to exclusively have processors larger than 100 qubits. Some of them are located in an IBM Quantum data center in upstate New York and deployed over the cloud for everyone's use — and some of them are dedicated, on-premises systems that support partners in the IBM Quantum Network. You can log into quantum.ibm.com to see which processors you have access to.

Each processor lists three performance metrics, which we discussed in the previous lesson, but as a reminder, they are: qubit count, EPLG, and CLOPS.

- Qubit count. This is self-explanatory: it's the number of total qubits available to use on a sole quantum processor. For a relatively large, utility-scale problem, you'll need to make sure you are using a processor with enough qubits to be able to tackle the problem. But the qubit count alone is not the only thing that matters.

- EPLG, or “errors per layered gate.” This is a measure of the quality of the qubits and quantum gates. It measures the average error each gate introduces in a circuit that entangles neighboring qubits in a chain of 100 qubits. You want this to be as small as possible.

- CLOPS, or “circuit layer operations per second.” This quantifies the speed of the processor. It measures how many layers of a certain benchmarking circuit called a quantum volume circuit a quantum processing unit (QPU) can execute per unit of time. The higher the number, the faster we can compute.

The importance of each of these metrics varies depending on the specific application, and in future lessons, we'll look at real examples to see how each of these factors can affect the outcome of a calculation.
---
## Software: Qiskit and Qiskit Runtime
To turn your quantum problem into instructions for a quantum computer, you'll use Qiskit, the open-source software development kit designed for work on quantum computers developed by IBM. There's also the Qiskit Ecosystem — a collection of software tutorials and functions that build or extend upon the core functionalities of Qiskit — and Qiskit Runtime — a quantum computing service and programming model that allows users to design and optimize their quantum workloads and execute them efficiently by using Qiskit Runtime Primitives.

A primitive is a small building block that you can use to design a larger circuit or job. The two primitives that are most important to us are the sampler and the estimator, which we'll discuss in more depth shortly.

With the recent release of Qiskit 1.0, Qiskit has become more performative and stable than ever before. So, for those of you just getting started, you came in at the perfect time! For those of you that are already familiar with Qiskit, you'll need to download and reinstall the newest version. For a full installation guide, visit the page over in Docs.

---

## Quantum Circuits
Now we're ready to discuss the foundation of quantum programs: quantum circuits. This section will only serve as a refresher — if you aren't familiar with quantum circuits, we recommend learning about them in more depth by visiting Lesson 03: Quantum Circuits in the Basic of quantum information course before continuing.

A quantum circuit is a network of quantum gates and measurements linked by wires that represent qubits, as shown below. Quantum circuits can be read like sheet music, from left to right, starting at time 0 on the left. Virtual qubits — those that haven't yet been assigned to a physical qubit on a processor — are listed in increasing order from top to bottom.

Gates are represented by different symbols on the wires of the involved qubit(s). Single-qubit gates — like a Hadamard gate, depicted below (the box with the H) — affect only the qubit whose wire it's placed on. Multi-qubit gates — like a CNOT gate, also shown below (the plus sign in the circle with a line connected to q0) — affect two or more qubits. In the depicted CNOT gate, the state of the q1 changes according to the state of q0. After all the gates are performed, we can measure the qubits, indicated by the black gates with the measurement symbol. The outcomes of the measurements are written onto classical registers, the double-lined “meas” bus below.



One important characteristic of a circuit is its depth. The depth of a quantum circuit is the minimum number of “layers” of quantum gates, executed in parallel, required to complete the circuit. Quantum gates can be executed in parallel (at the same time) whenever they don't have any qubits in common. But if two or more gates act of the same qubit then we can't perform them in parallel — they must be performed in two separate layers, one after the other.

There's another, less obvious, way to determine the depth of a circuit, by playing a sort of game. The rules are simple: starting from any qubit wire on the left, you must travel to the right and count the number of gates you encounter in your path. You may hop to a neighboring wire only when it is connected to your current wire by a multi-qubit gate. The goal is to maximize the number of gates you encounter along your path. This maximal number also happens to be the depth of the circuit.



---

## Designing a quantum circuit: Qiskit patterns
So, how do we go about designing and running a quantum circuit? The easiest way to understand a typical quantum computing workflow is through Qiskit patterns. Qiskit patterns are a conceptual framework allowing users to run quantum workloads by implementing certain steps with modular tooling. This allows quantum computing tasks to be performed by a powerful heterogenous (CPU/GPU/QPU) computing infrastructure. The steps can be performed as-a-service and can incorporate resource management, which allows for the seamless composability of new capabilities as they're developed.

Here are the main steps, which experienced Qiskit users will likely recognize.

Map. This step formalizes how we take a general problem we're interested in and figure out how to map it onto a quantum computer in the form of a quantum circuit.

Optimize. In this step we use Qiskit's transpiler to route and lay out the circuit onto an actual, physical qubit hardware. This includes translating the individual quantum gates into sequences of operations that are performed on the hardware as well as an optimization in the layout of the gates.

Execute. Qiskit Runtime primitives provide the interface to IBM Quantum hardware that allows transpiled circuits to run. This step also includes using error suppression and mitigation techniques, which can largely be abstracted away from the user.

Post-process. In this step the data from the quantum processor itself is processed, providing the user with useful results on the original problem. Basically, this encompasses any further analysis of the data acquired.

### Map
The Map step essentially asks the question: “How do I translate my problem into a quantum circuit that can reasonably be run on quantum hardware?” There's no doubt about it: mapping is a hard problem and an active area of research. There isn't a foolproof method that guarantees success, but there are recommended guidelines and examples of problems we already know how to map.

The first guideline is to let classical computers do whatever work they're better at. Tasks that are easy for classical computers probably won't benefit from a quantum computer. Quantum computers are for problems that are classically hard. Of course, if this is your very first time using Qiskit or a quantum computer, don't worry about finding a problem that is computationally complex. Break it down into smaller, bite-sized problems that you can learn to address before going straight for a utility-scale project.

Next, translate the outcomes for your problem that you want to measure or understand into an expectation value or a cost function. A cost function is a problem-specific function that defines the problem's goal as something to be minimized or maximized. It can be used to see how well a trial state or solution is performing with respect to that goal. This notion can be applied to various applications in chemistry, machine learning, finance, optimization, and so on — it doesn't necessarily matter from what subject you're approaching the problem.

Also keep in mind that the hardware you're going to be using has a specific topology, as we discussed in the hardware section. Some qubits are connected, and some aren't — you'll need to map your problem to a circuit that respects IBM quantum processors' heavy-hex topology.

For now, the most important thing to keep in mind is that this stage requires practice. You need to have a good understanding of not only your problem, but also the hardware capabilities — and we'll go through specific examples and use-cases in future lessons to see how to balance all these considerations.

### Optimize
Next, we'll need to choose a quantum processor that has enough qubits of high enough quality that we can run our quantum circuit. Make these decisions guided by the three metrics that we discussed in the hardware section: qubit count, EPLG, and CLOPS.

Then we optimize our circuit for the selected hardware. First, we need to lay out and route our circuit efficiently. Layout refers to mapping the virtual qubits in the circuit to the physical qubits on the processor. Routing refers to tweaking the circuit so that the connectivity between virtual qubits in the circuit matches the connectivity of the physical qubits on the processor. There are a couple of things to keep in mind during the layout and routing stage.

Not all qubits are connected. Some are very far away from one another on the chip, and we need to reduce or eliminate long-distance interactions wherever possible. You could apply a sequence of SWAP gates between neighboring qubits to move the qubits information around, but SWAP gates are costly and prone to errors, so there may be betters ways of doing this. Try to avoid too many costly SWAP gates.

Layout and routing are iterative processes. You can do it by hand, but there's also a Qiskit tool called mapomatic , which can make recommendations for a physical qubit layout based on approximate error rates. The transpiler (which we'll discuss shortly) can also make an informed suggestion.

Next, we can compose sequences of one-qubit gates acting on the same qubit into single gates — and we can also sometimes get rid of unnecessary gates or combination of gates. For example, some combinations of gates can be reduced to simpler combinations — and in fact, sometimes a combination of gates might equate to the identity operation, so we can simply eliminate them. You can do this automatically using the Qiskit transpiler — but you can also do it manually on a gate-by-gate basis if you want more control.

Once we've improved the circuit layout, routing and gate counts — either by hand or using the transpiler — we usually want to visualize our circuit to make sure the timing of all the gates makes sense. There is an argument you can flag in the transpiler to visualize the timeline of your circuit, and make sure everything is lined up the way you would expect.

### Qiskit Transpiler
As mentioned previously, the Qiskit Transpiler can be used to help in the early stages of the patterns workflow. Now let's dig into its capabilities in more detail. It can rewrite a given input circuit so that it matches the topology of a specific quantum device and optimize the circuit for execution and resilience against noise. It also rewrites a given circuit into basis gates of the specific quantum processor you have selected to use.

Qiskit has four built-in transpilation pipelines corresponding to different optimization levels, and unless you're already familiar with quantum circuit optimization, we recommend using one of them. By default, the transpilation process includes these six steps:

Initialization. This stage runs any initial passes that are required before we start embedding the circuit onto the backend. This typically involves unrolling custom instructions and converting the circuit to just single- and two-qubit gates.

Layout. This stage maps the virtual qubits in the circuit to the physical qubits on a backend. See Layout Stage for more details.

Routing. This stage runs after a layout has been applied and injects gates (such as swap gates) into the original circuit to make it compatible with the backend's connectivity. See Routing Stage for more details.

Translation. This stage translates the gates in the circuit to the target backend's basis set. See Translation Stage for more details.

Optimization. This stage runs the main optimization loop repeatedly until a condition (such as reaching a certain target depth) is reached. We have four different optimization levels to choose from, described below.

Scheduling. This stage is for any hardware-aware scheduling passes. At a high level, the scheduling can be thought of as inserting delays into the circuit to account for idle time on the qubits between the execution of instructions.

There are four optimization levels ranging from 0 to 3, where higher optimization levels take more time and computational effort but may yield a better circuit. Optimization level 0 is intended for device characterization experiments and, as such, only maps the input circuit to the constraints of the target backend, without performing any optimizations. Optimization level 3 spends the most effort to optimize the circuit. However, as many of the optimization techniques in the transpiler are based on heuristics, spending more computational effort doesn't always result in an improvement in the quality of the output circuit. If this is of further interest, please check out the transpiler pages in the Qiskit documentation.

### Error suppression
The first step in reducing errors in a circuit is optimizing the layout, routing, and minimizing the gate count, which we've already done, either using the transpiler or on our own. Now let's talk about some more sophisticated methods of error suppression.

Error suppression refers to a class of techniques that transform a circuit during compilation to minimize errors. It is distinct from error mitigation, which we'll discuss later in the “Execute” section below. The two most common forms of error suppression that we use are dynamical decoupling and Pauli twirling:

Dynamical decoupling is used to effectively cancel out some of the environmental noise introduced when a qubit sits idle. By applying a series of gates at specific times, you can make it so the noise that accumulates in one part of the idle period approximately cancels the noise in the other part.
Pauli twirling is a way of inserting random gates not to cancel the noise, as in dynamical decoupling, but to simplify the noise. By inserting random gates, it prevents the effects of different errors from building up as fast, and makes the noise easier to characterize, since it now has a stochastic nature. This method also forms the basis of a powerful error mitigation technique, which we'll discuss below.

### Execute
Now we're ready to execute the quantum program. The Qiskit Runtime primitives provide an interface to IBM Quantum hardware, and they also abstract error suppression and mitigation away from the user. There are two primitives to choose from: the Sampler and the Estimator.

Qiskit Runtime's Sampler runs the circuit multiple times on a quantum device, performing measurements on each run, and reconstructing the probability distribution from the recovered bit strings. The more runs (or shots) it performs, the more accurate the results will be, but this requires more time and quantum resources. Specifically, it calculates the probability of obtaining each possible standard basis state by measuring the state prepared by the circuit.

Qiskit Runtime's Estimator uses a complex algebraic process to estimate the expectation value on a real quantum device by breaking down the observable into a combination of other observables with known eigenbases.

The Execute step is also when we can select error mitigation strategy. Error mitigation refers to techniques that allow users to reduce circuit errors by modeling the device noise that was present at the time of execution. Typically, this results in quantum pre-processing overhead related to model training and classical post-processing overhead to mitigate errors in the raw results by using the generated model. In exchange for this overhead, we're able to get much more accurate results.

There are multiple techniques we can implement for error mitigation. We'll discuss three, in increasing order of resilience to errors, but also, consequently, in increasing order of computational cost. Be aware, however, that this is an active area of research — so we'll likely continue to invent new ones and improve upon old ones.

At resilience level 0, the transpiler doesn't do anything to your circuit.

At level 1, it introduces a method called Twirled Readout Error eXtinction (T-REX). T-REX uses Pauli twirling, as was discussed in the error suppression section. As mentioned, inserting random gates into the circuit can make even very complicated, difficult-to-model noise look stochastic, and much easier to account for or subtract out in post-processing.

At resilience level 2, Zero Noise Extrapolation (ZNE) is added. This is a popular technique that we've had a lot of recent success with. The idea behind ZNE might be a bit surprising — we actually add noise on top of what's already there! But this allows us to extrapolate in the reverse direction, to predict what the results would look like if there were less and less noise.

Adding noise can be accomplished in a few different ways. For instance, we can stretch out the gates to make them longer and thus, more prone to error, or run more gates that ultimately result in an identity operation, so the circuit doesn't change functionally but we purposefully sample more noise. You do have to do this for every circuit and every expectation value you want to keep track of, though — so you can see how it can end up being computationally expensive.

One specific type of ZNE is called Probabilistic Error Amplification (PEA). Once we've learned a noise model for a gate, PEA works by sampling errors from that noise model and deliberately injecting them into the circuit. This isn't available in Qiskit yet, but it will be later this year.

The final form of error mitigation we'll discuss is Probabilistic Error Cancellation (PEC). Instead of being at the 3rd resilience level, PEC is a special capability you must turn on manually in Qiskit, because the required computational resources don't scale very well compared to the other error mitigation techniques. You begin by learning about the noise that's affecting your circuit — run noise-learning or noise-characterization circuits for each unique layer of two-qubit gates in your circuit. These results let you describe the noise in terms of Pauli operators. Once you know these noise terms, you can modify your circuits so that they effectively have the opposite Pauli gates built in to cancel these noise channels. In some ways, the process is similar to the way noise-cancelling headphones work. However, this way of undoing the noise is very costly, with a time to run that grows quickly and exponentially in the number of gates, so it may not be the best choice for a very large circuit.

### Post-process
The post-process stage is where we visualize and analyze the output of our quantum circuit. There are a number of Qiskit tools available for you to do this, such as the visualization and quantum-info modules. We won't cover these here, but we'll see these modules in action as we dive into some application examples in future lessons.

---

## Conclusion
Hopefully this lesson gave you a whirlwind tour of the main considerations and workflow we use when we want to run a utility-scale quantum computation. It was packed with information, and much of it won't sink in until we see some actual examples where these theoretical concepts are put into practice. So, that's what the remainder of the course will be. After all, this course isn't called Quantum Computing in Practice for nothing!

Next time, we'll look at a specific example of how to use the Qiskit patterns workflow to design and run a quantum circuit that solves the classic problem from graph theory called MaxCut.